In [62]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx

import matplotlib.pyplot as plt

sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

from shapely.geometry import Point, LineString

In [63]:
# numpy flip test
A = np.matrix([[0, 0, 1], [0, 0, 0], [0, 0, 0]])
A

matrix([[0, 0, 1],
        [0, 0, 0],
        [0, 0, 0]])

In [64]:
np.transpose(A)

matrix([[0, 0, 0],
        [0, 0, 0],
        [1, 0, 0]])

In [65]:
# read in OD Matrix
res_folder = r"./input_folder_Feb12_v12_v2017"
# need to make sure the index and the column headers are the same dtype, so when you calculate the 
# transpose matrix you can add it back to the imported matrix
test_input_trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_Feb12_v12_v2017.csv"), index_col = 0)

In [66]:
test_input_trade_flows.index

Int64Index([4, 8, 10, 22, 25, 31, 36, 48, 54, 60, 63, 69, 89, 90, 91, 92, 93], dtype='int64')

In [67]:
test_input_trade_flows.columns

Index(['4', '8', '10', '22', '25', '31', '36', '48', '54', '60', '63', '69',
       '89', '90', '91', '92', '93'],
      dtype='object')

In [68]:
test_input_trade_flows.columns = test_input_trade_flows.columns.astype(int)

In [69]:
test_input_trade_flows.columns

Int64Index([4, 8, 10, 22, 25, 31, 36, 48, 54, 60, 63, 69, 89, 90, 91, 92, 93], dtype='int64')

In [70]:
test_input_trade_flows_transposed = test_input_trade_flows.transpose()

In [71]:
test_input_trade_flows_transposed.index

Int64Index([4, 8, 10, 22, 25, 31, 36, 48, 54, 60, 63, 69, 89, 90, 91, 92, 93], dtype='int64')

In [72]:
test_input_trade_flows_mirrored = test_input_trade_flows + test_input_trade_flows_transposed
test_input_trade_flows_mirrored

,4,8,10,22,25,31,36,48,54,60,63,69,89,90,91,92,93
4,0,0,0,0,0,0,3863181,0,0,0,480498,5501,0,0,0,0,0
8,0,0,0,0,0,555265,0,0,0,0,0,12032,0,0,0,0,0
10,0,0,0,0,0,55526,0,0,0,0,0,0,0,0,0,0,0
22,0,0,0,0,59210,0,73905,0,59210,59210,0,278,0,0,0,0,0
25,0,0,0,59210,0,349424,314295,0,0,0,29741,215702,0,0,0,0,0
31,0,555265,55526,0,349424,0,8290,250000,349424,349424,0,555,0,0,0,0,0
36,3863181,0,0,73905,314295,8290,0,0,314295,314295,0,250454,0,0,0,0,0
48,0,0,0,0,0,250000,0,0,0,0,0,0,0,0,0,0,0
54,0,0,0,59210,0,349424,314295,0,0,0,29741,215702,0,0,0,0,0
60,0,0,0,59210,0,349424,314295,0,0,0,29741,215702,0,0,0,0,0


In [73]:
scenario_var = "_Feb12_v12_v2017"
cost_attr = 'cost'
res_folder = r"./"
G_pickle  = os.path.join(res_folder, "asia_railways_ver3_Feb12_v12_v2017.pickle")

# Read in and process input data
edges_geography = r"./geographic_data_Feb12_v12/edges.shp"
edges_data = gpd.read_file(edges_geography)

# Output files
complete_volume_map = os.path.join(res_folder, "complete_equal_volume_map_Feb12_v12_v2017.csv")
trade_volume_map = os.path.join(res_folder, "trade_volume_map_Feb12_v12_v2017.csv")

# need this for now...but instead show just read in nodes from edges?
nodes_geography = r"./geographic_data_Feb12_v12/rail_nodes_Feb12_v12.csv"
nodes_data = pd.read_csv(nodes_geography, dtype = {'X': np.float64, 'Y': np.float64})
nodes_data['Point'] = [Point(x, y) for x, y in zip(nodes_data['X'],nodes_data['Y'])]
nodes_data['node_id'] = nodes_data['node_id'].astype(str)

OD_df = pd.read_csv(os.path.join(res_folder, "asia_railways_output_OD_matrix{}.csv".format(scenario_var)), index_col=0)
G = nx.read_gpickle(G_pickle)

# trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_Feb_v5.csv"))

In [74]:
#len(trade_flows)

In [75]:
edges_data

,source,target,link_id,length,time,name_x,geometry
0,38,69,1,1304,28.977778,Kashgar,"LINESTRING (75.99371 39.46769, 89.18467 42.94998)"
1,40,69,29,791,17.577778,Khorgos,"LINESTRING (80.41190 44.21607, 89.18467 42.94998)"
2,38,70,2,174,3.866667,Kashgar,"LINESTRING (75.99371 39.46769, 75.41076 40.57480)"
3,36,70,3,259,10.360000,Kara Suu,"LINESTRING (72.86264 40.70247, 75.41076 40.57480)"
4,36,49,104,22,0.880000,Kara Suu,"LINESTRING (72.86264 40.70247, 72.81063 40.52017)"
...,...,...,...,...,...,...,...
104,20,91,114,1560,47.640000,None,"LINESTRING (60.64593 25.29688, 72.92575 19.02477)"
105,12,66,19,623,26.520000,None,"LINESTRING (59.75000 32.04000, 59.21328 35.27869)"
106,45,63,88,1040,44.770000,None,"LINESTRING (59.61583 36.26611, 51.42322 35.69698)"
107,20,93,115,3000,91.620000,None,"LINESTRING (60.64593 25.29688, 60.76567 14.76131)"


In [76]:
nodes_data[:18]

,node_id,name,country,border,gauge_change,if_border_is_an_origin_or_destination,Name,X,Y,Point
0,1,Aktau,KAZ,False,False,False,Aktau,51.194236,43.650086,POINT (51.19423615 43.6500856)
1,2,Aktobe,KAZ,False,False,False,Aktobe,57.166930,50.283883,POINT (57.16692971 50.28388335)
2,3,Aktogal,KAZ,False,False,False,Aktogal,79.922600,46.851951,POINT (79.92260014999999 46.8519507)
3,4,Almaty,KAZ,False,False,False,Almaty,76.857735,43.229664,POINT (76.85773523 43.22966422)
4,5,Andijan,UBZ,True,False,False,Andijan,72.284590,40.814750,POINT (72.28458996000001 40.8147501)
5,6,Angren,UBZ,False,False,False,Angren,70.074230,41.008453,POINT (70.07422952 41.00845288)
6,7,Arys,KAZ,False,False,False,Arys,68.813051,42.433659,POINT (68.8130505 42.43365853)
7,8,Ashgabat,TKM,False,False,False,Ashgabat,58.316298,37.943921,POINT (58.31629783 37.94392081)
8,9,Atrek,TKM,False,False,False,Atrek,54.021612,37.414975,POINT (54.02161162 37.41497489)
9,10,Baku,AZE,True,False,True,Baku,49.866373,40.409438,POINT (49.86637319 40.40943762)


In [77]:
OD_df

,38,69,70,36,5,25,35,54,37,65,...,80,81,89,94,92,90,91,93,18b,68b
38,0.171170,100000.171170,100012.021170,100168.996170,100130.181170,100115.641170,100118.921170,100115.011170,100121.451170,100133.511170,...,400328.761171,400402.591171,300187.211170,400214.711170,500242.211170,500337.101170,300352.591171,300352.591171,100298.611170,100156.541171
69,100000.171170,0.171170,100206.001170,168.996170,130.181170,115.641170,118.921170,115.011170,121.451170,133.511170,...,300328.761171,300402.591171,200187.211170,300214.711170,400242.211170,400337.101170,200352.591171,200352.591171,298.611170,156.541171
70,100012.021170,100206.001170,0.171170,100037.176170,100075.991170,100090.531170,100105.811170,100109.721170,100116.161170,100128.221170,...,400323.471171,400397.301171,300181.921170,400209.421170,500236.921170,500331.811170,300347.301171,300347.301171,100359.661170,100151.251171
36,100168.996170,168.996170,100037.176170,0.171170,38.986170,53.526170,68.806170,72.716170,79.156170,91.216170,...,300286.466171,300360.296171,200144.916170,300172.416170,400199.916170,400294.806170,200310.296171,200310.296171,322.656170,114.246171
5,100130.181170,130.181170,100075.991170,38.986170,0.171170,14.711170,29.991170,33.901170,40.341170,52.401170,...,300247.651171,300321.481171,200106.101170,300133.601170,400161.101170,400255.991170,200271.481171,200271.481171,283.841170,75.431171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,500337.101170,400337.101170,500331.811170,400294.806170,400255.991170,400241.451170,400226.171170,400222.261170,400215.821170,400203.761170,...,200050.171170,100025.171170,200150.061170,100122.561170,95.061170,0.171170,400175.171170,200075.171170,400490.761170,400254.451171
91,300352.591171,200352.591171,300347.301171,200310.296171,200271.481171,200256.941171,200241.661171,200237.751171,200234.851171,200246.911171,...,400175.171170,300150.171170,400300.611171,500297.561170,400270.061170,400175.171170,0.171170,200100.171170,200497.331170,200196.221170
93,300352.591171,200352.591171,300347.301171,200310.296171,200271.481171,200256.941171,200241.661171,200237.751171,200234.851171,200246.911171,...,200075.171170,100050.171170,400225.061170,300197.561170,200170.061170,200075.171170,200100.171170,0.171170,200497.331170,200196.221170
18b,100298.611170,298.611170,100359.661170,322.656170,283.841170,269.301170,272.581170,268.671170,275.111170,287.171170,...,300473.501170,300547.331170,200340.871170,300368.371170,400395.871170,400490.761170,200497.331170,200497.331170,0.171170,310.201171


# Map network volume based on ALL least-cost trips

In [78]:
def tabulate_edges(route):
    edge_table = []
    for idx in range(0, len(route) - 1):
        edge_table.append([route[idx], route[idx+1]])
    return(edge_table)

"""
def get_route(route, nodes_data):
    all_nodes = []
    for x in route:
        try:
            cur_node = nodes_data.loc[nodes_data['node_id'] == x]
            all_nodes.append(Point(cur_node.X, cur_node.Y))
        except:
            pass
    if len(all_nodes) > 1:
        return(LineString(all_nodes))
    else:
        return(None)
"""

"\ndef get_route(route, nodes_data):\n    all_nodes = []\n    for x in route:\n        try:\n            cur_node = nodes_data.loc[nodes_data['node_id'] == x]\n            all_nodes.append(Point(cur_node.X, cur_node.Y))\n        except:\n            pass\n    if len(all_nodes) > 1:\n        return(LineString(all_nodes))\n    else:\n        return(None)\n"

In [116]:
all_res = []
complete_edges = []

count = 0

for origin, row in OD_df.iterrows():    
    for destination, value in row.items():
        try:
            origin = int(origin)
        except:
            pass
        try:
            destination = int(destination)
        except:
            pass
        
        count = count + 1
        
        if origin == 69:
            if destination == 49:
                print(origin, destination, value)
                print(nx.dijkstra_path(G, origin, destination, weight='cost'))
        
        list1 = nx.dijkstra_path(G, origin, destination, weight='cost')
        if 69 in list1:
            if 38 in list1:
                print(list1)
                
        route = nx.dijkstra_path(G, origin, destination, weight='cost')
        #print(route)
        edge_table = tabulate_edges(route)
        #print(edge_table)
        complete_edges = complete_edges + edge_table 
        #all_res.append([origin, destination, get_route(route, nodes_data)])


[38, 69]
[38, 69, 40, 4, 17, 7, 60, 6, 50, 25, 5, 36]
[38, 69, 40, 4, 17, 7, 60, 6, 50, 25, 5]
[38, 69, 40, 4, 17, 7, 60, 6, 50, 25]
[38, 69, 40, 4, 17, 7, 60, 54, 35]
[38, 69, 40, 4, 17, 7, 60, 54]
[38, 69, 40, 4, 17, 7, 60, 54, 37]
[38, 69, 40, 4, 17, 7, 60, 54, 37, 65]
[38, 69, 40, 4, 17, 7, 60, 54, 37, 65, 31]
[38, 69, 40, 4, 17, 7, 60, 54, 39]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43, 58]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43, 58, 55]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43, 58, 55, 45]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43, 58, 55, 45, 66]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43, 58, 55, 45, 66, 12]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43, 58, 55, 45, 66, 12, 76]
[38, 69, 40, 4, 17, 7, 60, 54, 39, 68, '68b', 43, 58, 55, 45, 66, 12, 76, 20]
[38, 69, 40, 4, 17, 7, 60, 54, 37, 65, 31, 42]
[38, 69, 40, 4, 17, 7, 60, 54, 37, 65, 31, 42, 59]


[81, 93, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54, 60, 7, 17, 4, 40, 69, 38]
[89, 42, 31, 65, 37, 54, 60, 7, 17, 4, 40, 69, 38]
[94, 89, 42, 31, 65, 37, 54, 60, 7, 17, 4, 40, 69, 38]
[92, 94, 89, 42, 31, 65, 37, 54, 60, 7, 17, 4, 40, 69, 38]
[90, 92, 94, 89, 42, 31, 65, 37, 54, 60, 7, 17, 4, 40, 69, 38]
[91, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54, 60, 7, 17, 4, 40, 69, 38]
[93, 20, 76, 12, 66, 45, 55, 58, 43, '68b', 68, 39, 54, 60, 7, 17, 4, 40, 69, 38]
['18b', 27, 74, 29, 56, 48, 87, 2, 53, 7, 17, 4, 40, 69, 38]
['68b', 68, 39, 54, 60, 7, 17, 4, 40, 69, 38]


In [97]:
count

8100

In [98]:
len(complete_edges)

78878

In [107]:
complete_edges[:16]

[[38, 69],
 [38, 70],
 [38, 69],
 [69, 40],
 [40, 4],
 [4, 17],
 [17, 7],
 [7, 60],
 [60, 6],
 [6, 50],
 [50, 25],
 [25, 5],
 [5, 36],
 [38, 69],
 [69, 40],
 [40, 4]]

In [109]:
complete = pd.DataFrame(complete_edges, columns=['o', 'd'])

In [113]:
complete.loc[complete['d']==69]

,o,d
0,38,69
2,38,69
13,38,69
23,38,69
32,38,69
...,...,...
76024,40,69
77108,40,69
77122,40,69
78196,40,69


In [100]:
complete['w'] = 1
complete_count = pd.DataFrame(complete.groupby(['o','d']).count())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [101]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [102]:
complete_count.loc[complete_count['o']=='69']

,o,d,w,geometry
17,69,21,12,"LINESTRING (89.18466843 42.94997926, 82.488012..."
21,69,40,163,"LINESTRING (89.18466843 42.94997926, 80.411899..."
185,69,38,88,"LINESTRING (89.18466843 42.94997926, 75.993707..."


In [103]:
complete_count.loc[complete_count['d']=='69']

,o,d,w,geometry
82,21,69,12,"LINESTRING (82.48801228000001 45.25034389, 89...."
83,38,69,88,"LINESTRING (75.99370752999999 39.46768501, 89...."
84,40,69,163,"LINESTRING (80.41189971999999 44.21606721, 89...."


In [104]:
complete_count

,o,d,w,geometry
0,1,10,308,"LINESTRING (51.19423615 43.6500856, 49.8663731..."
1,62,10,381,"LINESTRING (44.82710578 41.71513710000001, 49...."
2,67,10,244,"LINESTRING (52.96318620000001 40.02016264, 49...."
3,1,16,905,"LINESTRING (51.19423615 43.6500856, 55.1818477..."
4,53,16,926,"LINESTRING (60.31394877 47.04978092, 55.181847..."
...,...,...,...,...
201,92,90,71,"LINESTRING (71.56378993 34.10942691, 67.001097..."
202,89,94,215,"LINESTRING (69.19380137 34.50670823, 71.111711..."
203,92,94,142,"LINESTRING (71.56378993 34.10942691, 71.111711..."
204,90,92,71,"LINESTRING (67.00109753 24.89698664, 71.563789..."


In [49]:
complete_count.to_csv(complete_volume_map)

# Map specific routes with trade flows

In [50]:
# Instead I will be parsing through the trade flows DataFrame (Matrix)
complete_edges_trade = []

for idx in test_input_trade_flows_mirrored.iterrows():
    #print(idx[0])
    row = idx[0]
    #print(idx[1])
    for col,value in idx[1].items():
        #print(col)
        print(row,col,value)
        route = nx.dijkstra_path(G, row, col, weight = 'cost')
        if origin == 91:
            print(origin,destination,route)
        edge_table = tabulate_edges(route)
        [x.append(value) for x in edge_table]
        complete_edges_trade = complete_edges_trade + edge_table 





4 4 0
4 8 0
4 10 0
4 22 0
4 25 0
4 31 0
4 36 3863181
4 48 0
4 54 0
4 60 0
4 63 480498
4 69 5501
4 89 0
4 90 0
4 91 0
4 92 0
4 93 0
8 4 0
8 8 0
8 10 0
8 22 0
8 25 0
8 31 555265
8 36 0
8 48 0
8 54 0
8 60 0
8 63 0
8 69 12032
8 89 0
8 90 0
8 91 0
8 92 0
8 93 0
10 4 0
10 8 0
10 10 0
10 22 0
10 25 0
10 31 55526
10 36 0
10 48 0
10 54 0
10 60 0
10 63 0
10 69 0
10 89 0
10 90 0
10 91 0
10 92 0
10 93 0
22 4 0
22 8 0
22 10 0
22 22 0
22 25 59210
22 31 0
22 36 73905
22 48 0
22 54 59210
22 60 59210
22 63 0
22 69 278
22 89 0
22 90 0
22 91 0
22 92 0
22 93 0
25 4 0
25 8 0
25 10 0
25 22 59210
25 25 0
25 31 349424
25 36 314295
25 48 0
25 54 0
25 60 0
25 63 29741
25 69 215702
25 89 0
25 90 0
25 91 0
25 92 0
25 93 0
31 4 0
31 8 555265
31 10 55526
31 22 0
31 25 349424
31 31 0
31 36 8290
31 48 250000
31 54 349424
31 60 349424
31 63 0
31 69 555
31 89 0
31 90 0
31 91 0
31 92 0
31 93 0
36 4 3863181
36 8 0
36 10 0
36 22 73905
36 25 314295
36 31 8290
36 36 0
36 48 0
36 54 314295
36 60 314295
36 63 0
36 69 250454
3

In [51]:
complete_edges_trade

[[4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 39, 0],
 [39, 68, 0],
 [68, '68b', 0],
 ['68b', 43, 0],
 [43, 8, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 53, 0],
 [53, 16, 0],
 [16, 1, 0],
 [1, 10, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 37, 0],
 [37, 22, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 6, 0],
 [6, 50, 0],
 [50, 25, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 37, 0],
 [37, 65, 0],
 [65, 31, 0],
 [4, 17, 3863181],
 [17, 7, 3863181],
 [7, 60, 3863181],
 [60, 6, 3863181],
 [6, 50, 3863181],
 [50, 25, 3863181],
 [25, 5, 3863181],
 [5, 36, 3863181],
 [4, 17, 0],
 [17, 7, 0],
 [7, 53, 0],
 [53, 2, 0],
 [2, 87, 0],
 [87, 48, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [4, 17, 480498],
 [17, 7, 480498],
 [7, 60, 480498],
 [60, 54, 480498],
 [54, 39, 480498],
 [39, 68, 480498],
 [68, '68b', 480498],
 ['68b', 43, 480498],
 [43, 58, 480498],
 [58, 55, 480498],
 [55, 45, 480498],
 [

In [52]:
'''
#trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_v1.csv"))
complete_edges = []
for idx, row in trade_flows.iterrows():    
    
    try:
        origin = row['Node number']
        #print(origin)
        
            
        destination = row['Node Number']
        route = nx.bidirectional_shortest_path(G, origin, destination)
        if origin == 91:
            print(origin,destination,route)
        edge_table = tabulate_edges(route)
        [x.append(int(row['Trade volume'].replace(",", ""))) for x in edge_table]
        complete_edges = complete_edges + edge_table 
    except:
        pass
'''     

    


'\n#trade_flows = pd.read_csv(os.path.join(res_folder, "trade_flows_v1.csv"))\ncomplete_edges = []\nfor idx, row in trade_flows.iterrows():    \n    \n    try:\n        origin = row[\'Node number\']\n        #print(origin)\n        \n            \n        destination = row[\'Node Number\']\n        route = nx.bidirectional_shortest_path(G, origin, destination)\n        if origin == 91:\n            print(origin,destination,route)\n        edge_table = tabulate_edges(route)\n        [x.append(int(row[\'Trade volume\'].replace(",", ""))) for x in edge_table]\n        complete_edges = complete_edges + edge_table \n    except:\n        pass\n'

In [53]:
len(complete_edges_trade)

2312

In [54]:
complete_edges_trade

[[4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 39, 0],
 [39, 68, 0],
 [68, '68b', 0],
 ['68b', 43, 0],
 [43, 8, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 53, 0],
 [53, 16, 0],
 [16, 1, 0],
 [1, 10, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 37, 0],
 [37, 22, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 6, 0],
 [6, 50, 0],
 [50, 25, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 37, 0],
 [37, 65, 0],
 [65, 31, 0],
 [4, 17, 3863181],
 [17, 7, 3863181],
 [7, 60, 3863181],
 [60, 6, 3863181],
 [6, 50, 3863181],
 [50, 25, 3863181],
 [25, 5, 3863181],
 [5, 36, 3863181],
 [4, 17, 0],
 [17, 7, 0],
 [7, 53, 0],
 [53, 2, 0],
 [2, 87, 0],
 [87, 48, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [4, 17, 480498],
 [17, 7, 480498],
 [7, 60, 480498],
 [60, 54, 480498],
 [54, 39, 480498],
 [39, 68, 480498],
 [68, '68b', 480498],
 ['68b', 43, 480498],
 [43, 58, 480498],
 [58, 55, 480498],
 [55, 45, 480498],
 [

In [55]:
complete = pd.DataFrame(complete_edges_trade, columns=['o', 'd', 'w'])
complete_count = pd.DataFrame(complete.groupby(['o','d']).sum())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [56]:
complete_count

,o,d,w
0,1,10,0
1,1,16,0
2,1,71,0
3,2,53,250000
4,2,87,250000
...,...,...,...
113,93,81,0
114,94,89,0
115,94,92,0
116,68b,43,1760152


In [57]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on='d', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [58]:
complete_count.loc[complete_count['o']=='17']

,o,d,w,geometry
44,17,4,5821712,"LINESTRING (73.91684012 43.65288192, 76.857735..."
46,17,7,5821712,"LINESTRING (73.91684012 43.65288192, 68.813050..."


In [59]:
complete_count.loc[complete_count['d']=='17']

,o,d,w,geometry
11,4,17,5821712,"LINESTRING (76.85773523 43.22966422, 73.916840..."
12,7,17,5821712,"LINESTRING (68.8130505 42.43365853, 73.9168401..."


In [60]:
complete_count

,o,d,w,geometry
0,1,10,0,"LINESTRING (51.19423615 43.6500856, 49.8663731..."
1,67,10,55526,"LINESTRING (52.96318620000001 40.02016264, 49...."
2,1,16,0,"LINESTRING (51.19423615 43.6500856, 55.1818477..."
3,53,16,0,"LINESTRING (60.31394877 47.04978092, 55.181847..."
4,1,71,0,"LINESTRING (51.19423615 43.6500856, 52.8616865..."
...,...,...,...,...
113,92,94,0,"LINESTRING (71.56378993 34.10942691, 71.111711..."
114,90,81,0,"LINESTRING (67.00109753 24.89698664, 62.291348..."
115,93,81,0,"LINESTRING (60.7656735 14.76131335, 62.2913486..."
116,90,92,0,"LINESTRING (67.00109753 24.89698664, 71.563789..."


In [61]:
complete_count.to_csv(trade_volume_map)